In [2]:
# Import Dependencies
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
import requests
import json
from collections import Counter


In [21]:
# Import artist csv
data = pd.read_csv("data/artists.csv")
data.head()

,usless data,artist id
0,1:43,spotify:artist:39EHxSQAIaWusRqSI9xoyF
1,2:00 AM,spotify:artist:4tN3rZ7cChj4Wns2Wt2Nj6
2,2:15,spotify:artist:4HsOm6VNKZtGh8W8GhdNu4
3,2:54,spotify:artist:3LsQKoRgMc8VEkQn66jfAQ
4,4:20,spotify:artist:5KCG0FDMDPzQpxcohGUnyH


In [24]:
# Split artist column to get artist id 
data = data['artist id'].str.split(':', expand=True)
data.head()

,0,1,2
0,spotify,artist,39EHxSQAIaWusRqSI9xoyF
1,spotify,artist,4tN3rZ7cChj4Wns2Wt2Nj6
2,spotify,artist,4HsOm6VNKZtGh8W8GhdNu4
3,spotify,artist,3LsQKoRgMc8VEkQn66jfAQ
4,spotify,artist,5KCG0FDMDPzQpxcohGUnyH


In [43]:
# Create list 
artist_list = data[2].tolist()
len(artist_list)
# Temp list for testing Spotify API
temp_list= artist_list[:100]
print(temp_list)

['39EHxSQAIaWusRqSI9xoyF', '4tN3rZ7cChj4Wns2Wt2Nj6', '4HsOm6VNKZtGh8W8GhdNu4', '3LsQKoRgMc8VEkQn66jfAQ', '5KCG0FDMDPzQpxcohGUnyH', '5abblGojcZoe3zZBhJFBUi', '0oBPg2seHzVcAIOpdi1Ojj', '4CwN7lHeOBgAcyk7nNKSTJ', '6n4LC99DYefuU6icCPk1zU', '6pOCEbvtPkErkz3884ikYF', '1BYnP5HY7Ci0MFEHiDV1Ww', '6GW4TcHaVpWHrLp5tvLr6c', '5F1DQTsRoDRmlMWeQGi6jz', '6bMul6rmRS03x38tWKYifO', '0Qh5UGy4Y1hrZlClkIP7oG', '3w8DTEOpDhhdrOaJPOpSDD', '51kPCmCJ7rXClxKDc2r4RA', '5fJyMNVZGDGxsAxNLgoM7T', '2HIUtL0x7DtEU9cuQuO3Gn', '733SPLVnEyXs6GxUEnJ7bx', '6soPpJHlCtN6SY8pWlfbC6', '287576k08xwYGvRgcJSTRr', '4EbEUPsgjjmNsKjigP3R4u', '2NxTSd8rNDFhTQYDeb7pGj', '6bdOuFTmkGliFvCkdpH49Z', '3TTbkikkCAbUg2j5i9UrTV', '1mmehjf7eHA10uHMisZGJg', '0bw7sLDsNQTScMpmsNFukJ', '71WhWdsVNTLxsnfe8M3Peh', '7zug34wHklQjAqvDHAhqJ3', '3ZZHB1Xh2PM88QTMiqHB62', '1h2OVSKRdJHnVqyCKqCLkf', '0jsnWH4QCKcAMpR7vEhh9u', '6wBUn8gMP85n8dPu6LoUcF', '0kVQBzHzHP111Qt5rU7F0d', '5RzauMZQELIuit9D6Y2KKd', '5PwgSnrvg48DYvdsf2Y2ZD', '2cez574BwRD00PGEbNl4SH', '2PT695UML2

In [132]:
# Access token for Spotify API calls
# Enter your access_token given to you. Will have to figure out a way to import from js

access_token = "BQDIjDSlYMncceYFPKZeTuXcWF8ODHP9q88zlpB3zoOhIombaWanp2jT7yf8VWIUHIkUqaiqHxYmu3q_C0nofmuFh2reKdLRMsBxvBbifYAPHhZCd0YcFGOlm6FwKnIw7GgAQDqSrqAMk5WbshUvX27-gAHkcz3Jg0WP"

In [81]:

song_ids = []

for artist in temp_list:
    
#     Spotify API search endpoint returns list of tracks matching track title in order of popularity
    endpoint = f"https://api.spotify.com/v1/artists/{artist}/top-tracks?country=us"
    headers = {"Authorization":f"Bearer {access_token}"}

    response = requests.get(endpoint,headers=headers).json()
    
    try:
#         Grab up to ten top tracks from each atrist. Some artists don't have ten tracks to pull.
        for i in range(0,9):
            print(i ,response["tracks"][i]["id"])
            song_id = response["tracks"][i]["id"]
            song_ids.append(song_id)
            


        
    except: 
# #         Alert for any track that was not appended to lists
        print("Song not Found")


0 1vPk6ZTflnvm8LgtXzPDrT
1 6OpIhef0IbK0tkEU2m9c4T
2 36qxb23eWvnQiJWEMgWYO6
3 1FIlg68WTgn0xGL8vPy5Rj
4 7oBAMX1mPa2qiqmSunLirZ
5 0gXysVSXeGvRwEhcdKKDll
6 29cZPOOZ6uH6ipBhneNGwQ
7 5N0akr3xBzThgPYK7JMa7z
8 2wvHCpDbeA5iDHVNAOxb3A
0 35lyqJMy8RQL7XNeFxdWiO
Song not Found
0 5wy1pj7q13Adv1HdweimfP
1 5U9uNsWl0Ath8Z03srA6UC
Song not Found
0 0OfC5NzT1px3WbNcAMTa7G
1 2AdZ2FLQRh1XpGU04isbkN
2 6tt3r8X7VsSBhGj01aomaF
3 0U4urz5pBQVBTJchQP8oJU
4 27TnMOcm9qbxPYFMaiv2c9
5 7id4jwYsYzXSwPbUJ5mCoW
6 6kXGwdMOH9bXpkGLO4rPyS
7 5nYSIfnYS7bFTcZRgW5iSB
8 7MDscA0SNxzawHCU2ILyWt
0 42FzueZJJtYsrtSjSH1whB
1 02JLdxlJep7ryDDh2wy8dX
2 5gQ2jUakk252eBRuGL9mnm
3 1m7ZGGbYb3BWhevyOHOkWk
4 07H5qlncvSZTzAr2xlGOj8
5 4U7thXzuE7UDIT8HU4rW6u
6 31mzZrWwNSb7msy89RMK4P
7 3dnvCMkiRP40nmkXtToVOe
8 0VMnylz6b7Gyqcs556GpyY
0 34oIv6vgDReQ9kEiERPJaB
1 0glmMY96qUMGzLCUeahTrZ
2 0TcLDgtZ3pam1Jc4CuafSh
3 0xWRgJHHV2sIWc2gE7u9Ci
Song not Found
0 4uGtjkyzYZy0s9FTTyKumo
1 0ftJXRNb2HFpP7EFOfL4fC
2 4tkyVUleBVvJsdGY7hduZz
3 0R85Ud0h57s48qWJGCJ5hB
4 2Td

Song not Found
0 3RUytXAnGwi02QFkBvRUv3
Song not Found
0 7nhSO9GYru1AYREmIq2Too
1 5VhDYAIGxYBsc8gxFgpZs2
Song not Found
0 40F6hDk7Z7PJNua8SSErJH
1 5yrGsJoG4FsTg6luHlNGK2
2 0dY7CeoOMM3dhE7gqRwyXN
3 2ojddPZAy0NMCDBDptqM61
4 7J6FexHtHr5kaUUmdHtxZT
5 1HhFnrfVuiUaDdkRYuomsU
6 5WsaFV9NRTnX83nwpFsujG
7 494DebrXoSlc4k4l7lE8B0
8 3uLAGRF1JEIVmDTLdNXKQv
0 0ozmdM7J3BQfMlPwAFJiGQ
Song not Found
Song not Found
0 18N9hZG0MZryxpB4H36IyP
1 3NzLIbvq342cvzDlDH33RV
2 2vrUEPFeTnkeLqmG1JXiaZ
3 4q8CkKKnXQxiH4bTVKCm1I
4 19DXGGcr5hb3kh8BQtz4cE
5 6CHNlPWuSrXMpZPmB7gkFm
6 6ayjKIrJVuCLLbLtupBjzC
7 4PL0neV0CW44OxbbwVT9zl
8 1CRq2M7Tq4dl7Bwif65doB
0 1FWcnMIbxtGTiSq4bFU0a0
1 5yCeX9vpjBu8NiugXwKfBW
2 6WL8EGMLQvDTWfRklPrZjO
3 6CALmCM3Qh71iTuiOOl4KC
4 3PoLS6xDZQXNvZO7qMEeg1
5 5cZUTqqU5qJXoCq9gS1YmI
6 0Z1Ma78H0wYbc4Tp1tsPnG
7 0sDzqO1k5c7TV5TMVwTKY7
8 04DLd6K3BsprPuNFMqWbVj
Song not Found
0 0Jv1QRGJtGaBhbR2iMXq8t
1 1VgFg6EF9S8BuwLRCLz9QM
2 7A8dv2wqcNP6XnuLcNMd8g
3 3GJTaaXs41YlSBKEBU6sO7
4 59iJhqhcg2NAXpixaCdULl
5 1xn1uW4b

In [89]:
# Check number on songs pulled down from spotify call
len(song_ids)

650

In [88]:
# Check for dups
seen = set()
uniq = []
for x in song_ids:
    if x not in seen:
        uniq.append(x)
        seen.add(x)
len(uniq)

650

In [134]:
# List to Load into db
features = []

for song_id in song_ids:
    
#     Call Spotify API to collect Audio features data
    endpoint = f"https://api.spotify.com/v1/audio-features/{song_id}"
    headers = {"Authorization":f"Bearer {access_token}"}

    response = requests.get(endpoint,headers=headers).json()

#     print(response)

    
#     Append each feature
    features.append({
        "id": response["id"],
        "key": response["key"],
        "mode": response["mode"],
        "danceability": response["danceability"],
        "energy": response["energy"],
        "loudness": response["loudness"],
        "speechiness": response["speechiness"],
        "tempo": response["tempo"],
        "acousticness": response["acousticness"],
        "instrumentalness": response["instrumentalness"],
        "liveness": response["liveness"],
        "valence": response["valence"]
    })
    

In [136]:
print(len(features),features)

650 [{'id': '1vPk6ZTflnvm8LgtXzPDrT', 'key': 7, 'mode': 1, 'danceability': 0.641, 'energy': 0.665, 'loudness': -7.839, 'speechiness': 0.0263, 'tempo': 138.017, 'acousticness': 0.571, 'instrumentalness': 0.00107, 'liveness': 0.159, 'valence': 0.677}, {'id': '6OpIhef0IbK0tkEU2m9c4T', 'key': 7, 'mode': 1, 'danceability': 0.673, 'energy': 0.405, 'loudness': -9.323, 'speechiness': 0.0313, 'tempo': 155.959, 'acousticness': 0.718, 'instrumentalness': 0.000268, 'liveness': 0.122, 'valence': 0.652}, {'id': '36qxb23eWvnQiJWEMgWYO6', 'key': 7, 'mode': 1, 'danceability': 0.725, 'energy': 0.488, 'loudness': -9.696, 'speechiness': 0.0336, 'tempo': 124.093, 'acousticness': 0.247, 'instrumentalness': 0, 'liveness': 0.0801, 'valence': 0.456}, {'id': '1FIlg68WTgn0xGL8vPy5Rj', 'key': 11, 'mode': 1, 'danceability': 0.685, 'energy': 0.503, 'loudness': -7.174, 'speechiness': 0.0416, 'tempo': 93.923, 'acousticness': 0.514, 'instrumentalness': 2.78e-05, 'liveness': 0.0822, 'valence': 0.807}, {'id': '7oBAMX1mP

In [139]:
# Connect to db using sqlalchemy
db_uri = "sqlite:///music.db"
engine = create_engine(db_uri)

# create features table
# engine.execute('CREATE TABLE "features" ('
#                'id string NOT NULL,'
#                'key integer,'
#                'mode integer,'
#                'danceability integer,'
#                'energy integer,'
#                'loudness integer,'
#                'speechiness integer,'
#                'tempo integer,'
#                'acousticness integer,'
#                'instrumentalness integer,'
#                'liveness integer,'
#                'valence integer);')


In [138]:
# drop table if needed
# Connect to db using sqlalchemy
db_uri = "sqlite:///music.db"
engine = create_engine(db_uri)

# engine.execute("DROP TABLE features")

In [140]:
conn = sqlite3.connect('music.db')

c = conn.cursor()

for feature in features:
    c.execute("insert into features values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", 
              [feature['id'],feature['key'],feature['mode'],feature['danceability'],feature['energy'],
              feature['loudness'],feature['speechiness'],feature['tempo'],feature['acousticness'],feature['instrumentalness'],
              feature['liveness'],feature['valence']])
    conn.commit()
conn.close()

In [141]:
# verify table creation
result = engine.execute('SELECT * FROM "features"')
print(result.fetchall())

[('1vPk6ZTflnvm8LgtXzPDrT', 7, 1, 0.641, 0.665, -7.839, 0.0263, 138.017, 0.571, 0.00107, 0.159, 0.677), ('6OpIhef0IbK0tkEU2m9c4T', 7, 1, 0.673, 0.405, -9.323, 0.0313, 155.959, 0.718, 0.000268, 0.122, 0.652), ('36qxb23eWvnQiJWEMgWYO6', 7, 1, 0.725, 0.488, -9.696, 0.0336, 124.093, 0.247, 0, 0.0801, 0.456), ('1FIlg68WTgn0xGL8vPy5Rj', 11, 1, 0.685, 0.503, -7.174, 0.0416, 93.923, 0.514, 2.78e-05, 0.0822, 0.807), ('7oBAMX1mPa2qiqmSunLirZ', 7, 1, 0.608, 0.331, -10.692, 0.029, 138.131, 0.787, 0, 0.324, 0.709), ('0gXysVSXeGvRwEhcdKKDll', 0, 1, 0.709, 0.692, -7.687, 0.0241, 100.006, 0.589, 0, 0.0581, 0.927), ('29cZPOOZ6uH6ipBhneNGwQ', 7, 1, 0.546, 0.293, -10.918, 0.0272, 147.174, 0.915, 0, 0.166, 0.223), ('5N0akr3xBzThgPYK7JMa7z', 0, 1, 0.753, 0.55, -7.903, 0.0305, 113.018, 0.651, 0, 0.153, 0.663), ('2wvHCpDbeA5iDHVNAOxb3A', 6, 1, 0.66, 0.508, -8.368, 0.0283, 81.037, 0.274, 0, 0.121, 0.56), ('35lyqJMy8RQL7XNeFxdWiO', 5, 0, 0.707, 0.679, -7.522, 0.179, 126.041, 0.0278, 0, 0.102, 0.686), ('5wy1pj7

In [142]:
count = engine.execute('SELECT count(*) FROM features')
print(count.fetchall())

[(650,)]
